In [1]:
import pandas as pd
import joblib
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.stats import uniform
from google.colab import files

files.upload()

{}

## Latest Models' Scores:

In [3]:
last_scores = pd.read_csv('comparison_selected_vs_pca.csv')
last_scores

,Metric,Selected Features,PCA
0,Best Accuracy,0.901639,0.918033
1,Best AUC,0.963203,0.967532
2,Best Precision,0.843750,0.870968
3,Best Recall,0.964286,0.964286
4,Best F1,0.900000,0.915254


In [4]:
selected_data = pd.read_csv('/content/03_selected_features_dataset.csv')
pca_data = pd.read_csv('/content/04_heart_disease_pca.csv')
print(selected_data.head())
print(pca_data.head())

        age  trestbps      chol   thalach   oldpeak  sex_0  sex_1  cp_0  cp_1  \
0  0.948726  0.821446 -0.265040  0.015306  0.796300    0.0    1.0   1.0   0.0   
1  1.392002  1.723905  0.851214 -1.835388  0.011015    0.0    1.0   0.0   0.0   
2  1.392002 -0.682652 -0.349285 -0.910041  1.090782    0.0    1.0   0.0   0.0   
3 -1.932564 -0.081013  0.093004  1.645679  1.974227    0.0    1.0   0.0   0.0   
4 -1.489288 -0.081013 -0.875820  0.984717 -0.087146    1.0    0.0   0.0   1.0   

   cp_2  ...  restecg_2  exang_0  exang_1  slope_0  slope_1  ca_0  ca_2  \
0   0.0  ...        1.0      1.0      0.0      0.0      0.0   1.0   0.0   
1   0.0  ...        1.0      0.0      1.0      0.0      1.0   0.0   0.0   
2   0.0  ...        1.0      0.0      1.0      0.0      1.0   0.0   1.0   
3   1.0  ...        0.0      1.0      0.0      0.0      0.0   1.0   0.0   
4   0.0  ...        1.0      1.0      0.0      1.0      0.0   1.0   0.0   

   thal_1  thal_3  target  
0     0.0     0.0       0  
1     

## Defining parameter grid for grid search and parameter distribution for randomized search:

In [5]:
param_grid = [
    {
        'svm__kernel': ['linear'],
        'svm__C': [0.1, 1, 10, 100]
    },
    {
        'svm__kernel': ['rbf'],
        'svm__C': [0.1, 1, 10, 100],
        'svm__gamma': [0.001, 0.01, 0.1, 1]
    }
]


param_dist = [
    {
        'svm__kernel': ['linear'],
        'svm__C': uniform(loc=0.1, scale=100)
    },
    {
        'svm__kernel': ['rbf'],
        'svm__C': uniform(loc=0.1, scale=100),
        'svm__gamma': uniform(loc=0.001, scale=1)
    }
]

In [6]:

datasets = {
    'selected_data': selected_data,
    'pca_data': pca_data
}
pipeline = Pipeline([ ('svm', SVC(probability=True)) ])

results=[]
best_models = {} # Dictionary to store the best models temporary

for name , data in datasets.items():
  print(f"--- processing dataset: {name} ---")
  X = data.drop('target', axis=1)
  y = data['target']
  X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2 , random_state=42, stratify=y)

  print("Running GridSearchCV...")
  grid_search = GridSearchCV(pipeline, param_grid , cv=5, scoring='accuracy')
  grid_search.fit(X_train, y_train)

  best_grid_model = grid_search.best_estimator_
  y_pred_grid = best_grid_model.predict(X_test)
  y_proba_grid = best_grid_model.predict_proba(X_test)[:, 1]

  results.append({
      'dataset': name,
      'Search Type': 'GridSearch',
      'Best Parameters': grid_search.best_params_,
      'Best Score': grid_search.best_score_,
      'Test Accuracy': accuracy_score(y_test, y_pred_grid),
      'Test Precision': precision_score(y_test, y_pred_grid),
      'Test Recall': recall_score(y_test, y_pred_grid),
      'Test F1': f1_score(y_test, y_pred_grid),
      'Test AUC': roc_auc_score(y_test, y_proba_grid)
  })
  best_models[f'{name}_GridSearch'] = best_grid_model # Store the best grid search model


  print("Running RandomizedSearchCV...")
  random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)
  random_search.fit(X_train, y_train)

  best_random_model = random_search.best_estimator_
  y_pred_random = best_random_model.predict(X_test)
  y_proba_random = best_random_model.predict_proba(X_test)[:, 1]


  results.append({
      'dataset': name,
      'Search Type': 'RandomizedSearch',
      'Best Parameters': random_search.best_params_,
      'Best Score': random_search.best_score_,
      'Test Accuracy': accuracy_score(y_test, y_pred_random),
      'Test Precision': precision_score(y_test, y_pred_random),
      'Test Recall': recall_score(y_test, y_pred_random),
      'Test F1': f1_score(y_test, y_pred_random),
      'Test AUC': roc_auc_score(y_test, y_proba_random)
  })
  best_models[f'{name}_RandomizedSearch'] = best_random_model # Store the best randomized search model


# Display results
results_df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
print(results_df)

# Find the single best model based on Test Accuracy
best_row = results_df.loc[results_df['Test Accuracy'].idxmax()]
best_model_name_to_save = f"{best_row['dataset']}_{best_row['Search Type']}"
best_model_object_to_save = best_models[best_model_name_to_save]

# Save only the single best model
with open(f'{best_model_name_to_save}_best_model.pkl', 'wb') as f:
    pickle.dump(best_model_object_to_save, f)
print(f"Saved the overall best model: {best_model_name_to_save}_best_model.pkl ✔")

--- processing dataset: selected_data ---
Running GridSearchCV...
Running RandomizedSearchCV...
--- processing dataset: pca_data ---
Running GridSearchCV...
Running RandomizedSearchCV...
         dataset       Search Type  \
0  selected_data        GridSearch   
1  selected_data  RandomizedSearch   
2       pca_data        GridSearch   
3       pca_data  RandomizedSearch   

                                          Best Parameters  Best Score  \
0                  {'svm__C': 1, 'svm__kernel': 'linear'}    0.842772   
1  {'svm__C': 79.75429868602328, 'svm__kernel': 'linear'}    0.838520   
2                {'svm__C': 100, 'svm__kernel': 'linear'}    0.855187   
3  {'svm__C': 79.75429868602328, 'svm__kernel': 'linear'}    0.855187   

   Test Accuracy  Test Precision  Test Recall   Test F1  Test AUC  
0       0.852459        0.787879     0.928571  0.852459  0.949134  
1       0.868852        0.812500     0.928571  0.866667  0.957792  
2       0.901639        0.866667     0.928571  0.896

#📈Viewing Results:

In [14]:
results_df

,dataset,Search Type,Best Parameters,Best Score,Test Accuracy,Test Precision,Test Recall,Test F1,Test AUC
0,selected_data,GridSearch,"{'svm__C': 1, 'svm__kernel': 'linear'}",0.842772,0.852459,0.787879,0.928571,0.852459,0.949134
1,selected_data,RandomizedSearch,"{'svm__C': 79.75429868602328, 'svm__kernel': 'linear'}",0.838520,0.868852,0.812500,0.928571,0.866667,0.957792
2,pca_data,GridSearch,"{'svm__C': 100, 'svm__kernel': 'linear'}",0.855187,0.901639,0.866667,0.928571,0.896552,0.959957
3,pca_data,RandomizedSearch,"{'svm__C': 79.75429868602328, 'svm__kernel': 'linear'}",0.855187,0.901639,0.866667,0.928571,0.896552,0.959957


In [ ]:
results_df.to_csv('GridSearch_&_RandomSearch_Results.csv', index=False)

# Building a Full reusable Pipeline:
After identifying the best model and its best parameters, we make a pipeline that contains all preprocessing steps of the data and save it to be used for deployment.

In [2]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

data = pd.read_csv('/content/02_heart_disease_preprocessed.csv')

# Splitting the data
X = data.drop(columns=['target'])
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2 , random_state=42, stratify=y)

# Define Transformers
categorical_features = ['sex', 'cp', 'restecg', 'exang', 'slope', 'ca', 'thal']
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

encoder = OneHotEncoder(handle_unknown='ignore')
standardizer = StandardScaler()

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder', encoder, categorical_features),
        ('standardizer', standardizer, numerical_features)
    ],
    remainder='drop'
)

# Pipeline with tuned SVM
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.95)),
    ('svm', SVC(probability=True, random_state=42))
])

param_grid = [
    {
        'svm__kernel': ['linear'],
        'svm__C': [0.1, 1, 10, 100]
    },
    {
        'svm__kernel': ['rbf'],
        'svm__C': [0.1, 1, 10, 100],
        'svm__gamma': [0.001, 0.01, 0.1, 1] # gamma is mostly for 'rbf' kernel
    }
]

# Grid Search set up
search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2)
search.fit(X_train, y_train)

# Use the BEST pipeline found by the search
print("Best parameters found: ", search.best_params_)

# 'search' now acts as the best possible pipeline
best_pipeline = search.best_estimator_

#  Evaluate and Save the BEST pipeline
accuracy = best_pipeline.score(X_test, y_test)
print(f"Tuned Pipeline Accuracy: {accuracy:.4f}")

joblib.dump(best_pipeline, 'final_tuned_pipeline.pkl')
print("Tuned Pipeline Saved as final_tuned_pipeline.pkl")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters found:  {'svm__C': 100, 'svm__gamma': 0.001, 'svm__kernel': 'rbf'}
Tuned Pipeline Accuracy: 0.8689
Tuned Pipeline Saved as final_tuned_pipeline.pkl


#⭐Observation:
Accuracy dropped  (`0.90` -> `0.87`) because  The `0.9016` score was the average performance on validation folds of the training data, while the `0.8689` score is the final performance on unseen test data.

It indicates that the model is generalizing well, with only a minor, expected drop in performance on completely new data.

# Prediction Testing

In [13]:

def load_model(file_path):

  with open(file_path, 'rb') as file:
    model = pickle.load(file)
  return model

def load_pca_object(file_path):
  """Loads a pickled PCA object from the given file path."""
  with open(file_path, 'rb') as f:
    pca_object = pickle.load(f)
  return pca_object

def load_preprocessor(file_path):
  """Loads a pickled preprocessor object from the given file path."""
  with open(file_path, 'rb') as f:
    preprocessor_object = pickle.load(f)
  return preprocessor_object


def predict_heart_disease(patient_data, model_path, pca_path=None, preprocessor_path=None, selected_features=None):
  """
  Predicts the likelihood of heart disease for a patient using a trained model.

  Args:
    patient_data: A pandas DataFrame or array-like object containing the patient's features
                  in their original format (before preprocessing).
    model_path: The file path to the pickled trained model.
    pca_path: The file path to the pickled PCA transformer object, if the model was trained on PCA data.
    preprocessor_path: The file path to the pickled preprocessor object (ColumnTransformer/Pipeline).
    selected_features: A list of column names representing the features selected after initial preprocessing.

  Returns:
    A numpy array containing the predictions (0 for no heart disease, 1 for heart disease).
    If the model supports probability predictions, it will return the probability of the positive class.
  """
  try:

    model = load_model(model_path)

    # --- Preprocess the new patient data ---
    patient_data_processed = patient_data
    if preprocessor_path:
        loaded_preprocessor = load_preprocessor(preprocessor_path)
        patient_data_processed = loaded_preprocessor.transform(patient_data)

    # Columns after encoding and standardizing:
    processed_columns = [
        'age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'sex_0', 'sex_1' , 'cp_0', 'cp_1', 'cp_2', 'cp_3', 'fbs_0',
        'fbs_1','restecg_0', 'restecg_1', 'restecg_2', 'exang_0', 'exang_1', 'slope_0', 'slope_1', 'slope_2',
        'ca_0', 'ca_1', 'ca_2', 'ca_3', 'thal_1', 'thal_2', 'thal_3'
    ]

    # If the preprocessor output is a numpy array, convert it to DataFrame with known column names
    if not isinstance(patient_data_processed, pd.DataFrame):
         # Ensure the number of columns in the processed data matches the expected number
         if patient_data_processed.shape[1] != len(processed_columns):
             print(f"Error: Mismatch in number of processed features. Expected {len(processed_columns)}, got {patient_data_processed.shape[1]}.")
             print("Please verify the preprocessing steps and the 'processed_columns' list.")
             return None, None
         patient_data_processed = pd.DataFrame(patient_data_processed, columns=processed_columns)

    # Reindex to handle potential missing columns in new data not seen in training
    patient_data_processed = patient_data_processed.reindex(columns=processed_columns, fill_value=0)


    # --- Apply Feature Selection ---
    if selected_features is not None:
        # Ensure the selected_features list is a subset of processed_columns
        if not all(feature in processed_columns for feature in selected_features):
             print("Error: Selected features are not all present in the processed columns.")
             print("Please verify the 'selected_features' list.")
             return None, None

        patient_data_selected = patient_data_processed[selected_features]
    else:
        # If no selected features are provided, use the processed data as is
        patient_data_selected = patient_data_processed
    # --- End of Feature Selection ---


    # If a PCA path is provided, load and apply the PCA transformation
    if pca_path:
      loaded_pca = load_pca_object(pca_path)
      # Apply PCA transformation to the selected data
      patient_data_transformed = loaded_pca.transform(patient_data_selected)
      prediction = model.predict(patient_data_transformed)
      if hasattr(model, 'predict_proba'):
           probability = model.predict_proba(patient_data_transformed)[:, 1]
           return prediction, probability
      else:
           return prediction, None


    # Otherwise, predict directly with the loaded model (this case is less likely if PCA was used)
    prediction = model.predict(patient_data_selected) # Predict with selected data
    if hasattr(model, 'predict_proba'):
        probability = model.predict_proba(patient_data_selected)[:, 1]
        return prediction, probability
    else:
        return prediction, None


  except FileNotFoundError:
    print(f"Error: File not found at {model_path}, {pca_path}, or {preprocessor_path}")
    return None, None
  except Exception as e:
    print(f"An error occurred during prediction: {e}")
    return None, None

# Dummy Data
new_patient_data = {
    'age': [63],
    'sex': [1],
    'cp': [3],
    'trestbps': [145],
    'chol': [233],
    'fbs': [1],
    'restecg': [0],
    'thalach': [150],
    'exang': [0],
    'oldpeak': [2.3],
    'slope': [0],
    'ca': [0],
    'thal': [1]
}
new_patient_df = pd.DataFrame(new_patient_data)

best_model_file = '/content/pca_data_GridSearch_best_model.pkl'
pca_transformer_file = '/content/pca_transformer_opt.pkl'
preprocessor_file = '/content/preprocessor.pkl'

selected_features_list = [
    'age', 'trestbps', 'chol', 'thalach', 'oldpeak',
    'sex_0', 'sex_1',
    'cp_0', 'cp_1', 'cp_2', 'cp_3',
    'restecg_0', 'restecg_2',
    'exang_0', 'exang_1',
    'slope_0', 'slope_1',
    'ca_0', 'ca_2',
    'thal_1', 'thal_3'

]


prediction, probability = predict_heart_disease(new_patient_df, best_model_file, pca_path=pca_transformer_file, preprocessor_path=preprocessor_file, selected_features=selected_features_list)

if prediction is not None:
     print("Prediction:", prediction)
     if probability is not None:
         print("Probability of heart disease:", probability)

Prediction: [0]
Probability of heart disease: [0.27494775]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


## Training Summary:

After training SVM models on both the selected features dataset and the PCA-transformed dataset, and tuning hyperparameters using both Grid Search and Randomized Search, the results are summarized below:

| Dataset         | Search Type      | Best Parameters                                       | Best Score (Cross-Validation) | Test Accuracy | Test Precision | Test Recall | Test F1   | Test AUC  |
|-----------------|-------------------|-------------------------------------------------------|-------------------------------|---------------|----------------|-------------|-----------|-----------|
| selected_data   | GridSearch        | {'svm__C': 1, 'svm__kernel': 'linear'}                | 0.8428                        | 0.8525        | 0.7879         | 0.9286      | 0.8525    | 0.9491    |
| selected_data   | RandomizedSearch  | {'svm__C': 79.75, 'svm__kernel': 'linear'}            | 0.8385                        | 0.8689        | 0.8125         | 0.9286      | 0.8667    | 0.9578    |
| pca_data        | GridSearch        | {'svm__C': 100, 'svm__kernel': 'linear'}              | 0.8552                        | 0.9016        | 0.8667         | 0.9286      | 0.8966    | 0.9599    |
| pca_data        | RandomizedSearch  | {'svm__C': 79.75, 'svm__kernel': 'linear'}            | 0.8552                        | 0.9016        | 0.8667         | 0.9286      | 0.8966    | 0.9599    |

**Key Findings:**

*   The models trained on the **PCA-transformed data** generally achieved slightly higher test accuracy compared to the models trained on the selected features.
*   Both **Grid Search and Randomized Search** yielded similar performance metrics on the test set for the PCA data.
*   The overall **best performing model** based on Test Accuracy is the **SVM model trained on the PCA-transformed data with hyperparameters found by either Grid Search or Randomized Search**. This model achieved a Test Accuracy of 0.9016.
*   The best parameters for the top-performing models utilized a **linear kernel** with a high C value.

## Building and Evaluating a Full Pipeline:

A full reusable pipeline was built containing preprocessing steps (encoding and standardizing), PCA dimensionality reduction, and the tuned SVM model. This pipeline was trained on the training data and evaluated on the unseen test set.

The final tuned pipeline achieved a Test Accuracy of **0.8689**.

**Observation on Accuracy Drop:**

The accuracy on the final tuned pipeline (**0.8689**) is lower than the best test accuracy observed during the initial model comparison (**0.9016**). This difference is expected and often occurs for the following reasons:

*   The `0.9016` score was the test set performance of a model trained on pre-selected/pre-PCA'd data, potentially with some data leakage or specific tuning on that already-transformed data.
*   The `0.8689` score is the performance of the *entire pipeline* (including preprocessing and PCA) evaluated on a completely unseen test set. This provides a more realistic estimate of how the model will perform on new, raw data.

This final tuned pipeline has been saved for future use in making predictions on new data and for deployment.